In [5]:
import pandas as pd
import numpy as np

from skrub import TableVectorizer
import xgboost as xgb
from sklearn.pipeline import Pipeline

import holidays

In [6]:
# Import the files
df_train = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/train.parquet")
df_test = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/final_test.parquet")

In [7]:
# Extract the date feature on different time scales :

fr_holidays = holidays.France()

def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour

    # creation of a binary varible depicting if day in weekend
    X["is_weekend"] = np.where(X["weekday"] + 1 > 5, 1, 0)

    # Add a feature to indicate if the day is a holiday in France
    X["is_holiday"] = X["date"].apply(lambda d: 1 if d in fr_holidays else 0)

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

df_train = _encode_dates(df_train)
df_test = _encode_dates(df_test)


In [8]:
df_train.head()

,counter_id,counter_name,site_id,site_name,bike_count,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count,year,month,day,weekday,hour,is_weekend,is_holiday
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,2020,9,1,1,2,0,0
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147,2020,9,1,1,3,0,0
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,2020,9,1,1,4,0,0
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.609438,2020,9,1,1,15,0,0
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.302585,2020,9,1,1,18,0,0


In [9]:
df_test.head()

,counter_id,counter_name,site_id,site_name,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,year,month,day,weekday,hour,is_weekend,is_holiday
0,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2021,9,10,4,1,0,0
1,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2021,9,10,4,13,0,0
2,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2021,9,10,4,17,0,0
3,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2021,9,10,4,19,0,0
4,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2021,9,10,4,22,0,0


In [10]:
df_train.describe()

,site_id,bike_count,counter_installation_date,latitude,longitude,log_bike_count,year,month,day,weekday,hour,is_weekend,is_holiday
count,4.968270e+05,496827.000000,496827,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000
mean,1.053450e+08,60.191475,2019-04-04 07:24:35.245911,48.854343,2.345479,3.079917,2020.679846,6.556904,15.458226,2.992172,11.502730,0.283354,0.029564
min,1.000070e+08,0.000000,2013-01-18 00:00:00,48.826360,2.265420,0.000000,2020.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000475e+08,5.000000,2018-11-29 00:00:00,48.840801,2.314440,1.791759,2020.000000,4.000000,8.000000,1.000000,6.000000,0.000000,0.000000
50%,1.000562e+08,29.000000,2019-11-06 00:00:00,48.852090,2.353870,3.401197,2021.000000,7.000000,15.000000,3.000000,12.000000,0.000000,0.000000
75%,1.000563e+08,79.000000,2019-12-11 00:00:00,48.864610,2.375870,4.382027,2021.000000,9.000000,23.000000,5.000000,18.000000,1.000000,0.000000
max,3.000147e+08,1302.000000,2020-11-29 00:00:00,48.891720,2.409690,7.172425,2021.000000,12.000000,31.000000,6.000000,23.000000,1.000000,1.000000
std,3.210346e+07,87.590566,NaN,0.018554,0.038026,1.659899,0.466536,3.423834,8.851485,1.995015,6.920936,0.450627,0.169380


In [11]:
df_test.describe()

,site_id,counter_installation_date,latitude,longitude,year,month,day,weekday,hour,is_weekend,is_holiday
count,5.144000e+04,51440,51440.000000,51440.000000,51440.0,51440.000000,51440.000000,51440.000000,51440.000000,51440.000000,51440.0
mean,1.073050e+08,2019-05-05 04:44:51.881804,48.854275,2.344642,2021.0,9.466116,15.090921,3.081532,11.491660,0.308865,0.0
min,1.000070e+08,2013-01-18 00:00:00,48.826360,2.265420,2021.0,9.000000,1.000000,0.000000,0.000000,0.000000,0.0
25%,1.000475e+08,2018-11-29 00:00:00,48.839770,2.311790,2021.0,9.000000,10.000000,1.000000,6.000000,0.000000,0.0
50%,1.000563e+08,2019-11-06 00:00:00,48.852090,2.353870,2021.0,9.000000,15.000000,3.000000,11.000000,0.000000,0.0
75%,1.000563e+08,2019-12-12 00:00:00,48.864610,2.375870,2021.0,10.000000,21.000000,5.000000,17.000000,1.000000,0.0
max,3.000147e+08,2020-11-29 00:00:00,48.891720,2.409690,2021.0,10.000000,30.000000,6.000000,23.000000,1.000000,0.0
std,3.738839e+07,NaN,0.018607,0.038257,0.0,0.498855,7.716252,2.042702,6.903709,0.462030,0.0


In [12]:
X_train = df_train.drop(columns=["bike_count", "log_bike_count"])
y_train = df_train["log_bike_count"]

X_test = df_test.copy()

In [13]:
# Preprocessing pipeline
pipeline = Pipeline(
    steps=[
        ('preprocessor', TableVectorizer()),
        ('model', xgb.XGBRegressor()),  # Note the parentheses to instantiate the model
    ]
)

In [14]:
# Fit Pipeline to Training Data
pipeline.fit(X_train, y_train)

# Make Predictions on Test Data
y_predictions = pipeline.predict(X_test)


In [15]:
print(y_predictions)

[0.38414693 1.5921499  2.105191   ... 5.185658   4.768595   3.617191  ]


In [16]:
pd.DataFrame(y_predictions, columns=["log_bike_count"]).reset_index().rename(
    columns={"index": "Id"}
).to_csv("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/predictions_option_2_vsimple.csv", index=False)